# Libirary import

In [ ]:
""" os """
import os

""" torch """
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torchvision.transforms import Compose, Resize, ToTensor
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler
from torch.utils.data.sampler import SubsetRandomSampler
from torch import Tensor

"""glob"""
from glob import glob

""" tqdm """
import time
from tqdm import tqdm

"""Pandas"""
import pandas as pd

""" numpy """
import numpy as np
from numpy import argmax
from PIL import Image

"""JSON"""
import json

"""sklearn"""
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler

"""seaborn"""
import seaborn as sns

"""scipy"""
from scipy import io
from scipy import signal
from scipy.fft import fft, ifft,fftfreq
from scipy import stats

"""time"""
import time

"""PIL"""
from PIL import Image

import re
import shutil
import random
import matplotlib.pyplot as plt
import scipy

from IPython.display import Image

import imageio
import easydict

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

# Path init

In [ ]:
import sys
print(sys.version)
sys.path.append('/Function')
from preprocessing import get_x_y_pairs, preprocessing 
from Train_classification import Model_Train, Model_Test,Model_Test_Youden_Index, T_SNE

In [ ]:
work_path = "" # Our data paths, it is not opened
os.chdir(work_path)
os.listdir()

# Hyper parameters

In [ ]:
args = easydict.EasyDict({
    #init
    "Task"              :"Time series classification",
    "Dataset"           :"GSD patients data", 
    "Patient"           :'more then 14 days patients',
    "Data_Balanced"     :"Unknown",
    "Filtering"         :"None",
    "Normalization"     :"RobustScaler",
    "Loss"              :"BCE loss",
    "Preprocess"        :"",
    "Model_name"        :"Patch TST",
    "Hypo_threshold"    :"80",
    "basic_path"        :"", # result save path
    
    #hyper parameters
    "seed"              :1,
    "lr"                :0.001,
    "batch_size"        :128, # 4096
    "test_batch_size"   :1,
    "window_size"       :48, # 12 hours
    "forcast_size"      :4, # 1 hour
    "epochs"            :100,
    "no_cuda"           :False,
    "log_interval"      :100,
    
})

# Set the seed and GPU

In [ ]:
torch.manual_seed(args.seed)
use_cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


kwargs = {'num_workers':0,'pin_memory':True} if use_cuda else {}

!nvcc --version
#https://pytorch.org/get-started/previous-versions/
print('--------------------------------------')
print('현재 torch 버전:',torch.__version__)
print('학습을 진행하는 기기:',device)
print('cuda index:', torch.cuda.current_device())
print('gpu 개수:', torch.cuda.device_count())
print('graphic name:', torch.cuda.get_device_name())

# Model load

In [ ]:
import sys
sys.path.append('/Model')
from N_Linear import LTSF_NLinear
from D_Linear import LTSF_DLinear
from TSMixer import TSMixer
from PatchTST import PatchTST

from Model_performance import model_performance_forecasting

# 1. TS Mixer

In [ ]:
input = torch.randn(256, args.window_size, 7) 
target = torch.randn(256, args.forcast_size, 7)

model = TSMixer(sequence_length=args.window_size, prediction_length=1, input_channels=7, output_channels=7, num_blocks=8)

output = model(input) 

output.shape

# 2. M Linear

In [ ]:
input = torch.randn(256, args.window_size, 7) 
target = torch.randn(256, args.forcast_size, 7)

model = LTSF_NLinear(window_size=args.window_size, forcast_size=1, individual=False, feature_size=7)

output = model(input)
output.shape

# 3. Patch TST

In [ ]:
input = torch.randn(256, args.window_size, 7) 
target = torch.randn(256, args.forcast_size, 7)

model = PatchTST(channel = 7, window_size=args.window_size, forcast_size=1)

output = model(input) 
output.shape

# Classification model

In [ ]:
class Classification_model(nn.Module):
    
    def __init__(self, oup:int=7, class_n:int=1):
        super(Classification_model,self).__init__()
        
        #self.TS_model = TSMixer(sequence_length=args.window_size, prediction_length=1, input_channels=7, output_channels=7, num_blocks=8)
        #self.TS_model = LTSF_NLinear(window_size=args.window_size, forcast_size=1, individual=False, feature_size=7)
        self.TS_model = PatchTST(channel = 7, window_size=args.window_size, forcast_size=1)
        
        
        self.linear = nn.Linear(oup, class_n)
        
        
    def forward(self,x):
        
        out = self.TS_model(x) # B 1 7
        out = self.linear(out)

        return out

In [ ]:
input = torch.randn(256, args.window_size, 7) 

model = Classification_model()

output = model(input) 
output.shape

# Custom Dataset

In [ ]:
class CustomDataset(Dataset):
    
    def __init__(self, data_indexs, data_df, transform=None):
        self.data_indexs = data_indexs
        self.data_df     = data_df
        self.transform = transform
    
    def __getitem__(self, idx):
        
        start = self.data_indexs[idx]
        X  = self.data_df[start:start+args.window_size] # window
        Y  = self.data_df[start+args.window_size:start+args.window_size+args.forcast_size] # forecast
        
        
        """data"""
        year      = np.expand_dims(np.array(X['Year']),axis=1)
        month     = np.expand_dims(np.array(X['Month']),axis=1)
        day       = np.expand_dims(np.array(X['Day']),axis=1)
        minutes   = np.expand_dims(np.array(X['Minutes']),axis=1)
        min_sin   = np.expand_dims(np.array(X['Minutes_sin']),axis=1)
        min_cos   = np.expand_dims(np.array(X['Minutes_cos']),axis=1)
        

        glu       = np.expand_dims(np.array(X['Glucose_level_original']),axis=1)
        
        data = np.concatenate((year, month, day, minutes, min_sin, min_cos, glu),axis=1)
        
        
        """get label"""
        glu_original_y = np.array(Y['Glucose_level_original'])
        
        
        #label = glu_original_y[0] # 15m
        #label = glu_original_y[1] # 30m
        #label = glu_original_y[2] # 45m
        label = glu_original_y[3] # 60m
        
        if label <= float(args.Hypo_threshold):
            label = int(1)
        else:
            label = int(0)
            
        
        
        return data, label
    
    def __len__(self):
        return len(self.data_indexs)


# Data paths

In [ ]:
data_paths = os.listdir()
data_paths
len(data_paths)

# Patients selection(more than 14 days)

In [ ]:
new_data_paths = []

for i in range(0,len(data_paths),1):
    data_path = data_paths[i]
    df = pd.read_csv(data_path, engine='python', encoding='utf-8')
    
    day=np.round(len(df)/(4*24), 1)
    if day < 14:
        pass
    else:
        new_data_paths.append(data_path)

In [ ]:
len(new_data_paths)
data_paths = new_data_paths
len(data_paths)

# Experiment

In [ ]:
def check_single_class(loader):
    # Initialize a variable to track if only label 0 is found
    only_zero_label = True

    # Iterate over the DataLoader
    for _, labels in loader:
        # Check if there are any labels that are not 0
        if torch.any(labels != 0):
            only_zero_label = False
            break

    return only_zero_label

In [ ]:
no_hypo = []

for k in range(0,len(data_paths),1):
    
    
    """init"""
    data_path           = data_paths[k]
    args.experiment_num = k
    df                  = pd.read_csv(data_path)
    
    """make folder"""
    result_save_path = args.basic_path + args.Model_name +'/'+str(args.experiment_num)

    folder_path = result_save_path
    try:
        if not(os.path.isdir(folder_path)):
            os.makedirs(os.path.join(folder_path))
    except OSError as e:
        if e.errno != errno.EEXIST:
            print("Failed to create directory!!!!!")
            raise
    args.fold_path = folder_path
            
            
    """---------------------------------------------Dataset split---------------------------------------------"""
    train_df, valid_df, test_df = preprocessing(df, RobustScaler())
    print(len(train_df), len(valid_df), len(test_df))
    
    """---------------------------------------------Make pair---------------------------------------------"""
    train_indexs   = get_x_y_pairs(train_df, args.window_size, args.forcast_size)
    valid_indexs   = get_x_y_pairs(valid_df, args.window_size, args.forcast_size)
    test_indexs    = get_x_y_pairs(test_df, args.window_size, args.forcast_size) # 전체 시간대

    print(len(train_indexs),len(valid_indexs),len(test_indexs))
    
    
    """---------------------------------------------Custom dataset---------------------------------------------"""
    train_dataset      = CustomDataset(train_indexs, train_df, transforms.Compose([transforms.ToTensor()]))
    validation_dataset = CustomDataset(valid_indexs, valid_df, transforms.Compose([transforms.ToTensor()]))
    test_dataset       = CustomDataset(test_indexs, test_df, transforms.Compose([transforms.ToTensor()])) # 전체 시간대
    

    print(len(train_dataset),len(validation_dataset), len(test_dataset))
    
    """---------------------------------------------Data Loader---------------------------------------------"""
    """Train"""
    args.train_loader = torch.utils.data.DataLoader(
        dataset=train_dataset,
        batch_size = args.batch_size,
        shuffle = False, 
        **kwargs
    )

    """Validation"""
    args.validation_loader = torch.utils.data.DataLoader(
        dataset=validation_dataset,
        batch_size = args.batch_size,
        shuffle = False,
        **kwargs
    )

    """Test"""
    args.test_loader = torch.utils.data.DataLoader(
        dataset=test_dataset,
        batch_size = args.test_batch_size,
        shuffle = False,
        **kwargs
    )

    print("Length of the train_loader:", len(args.train_loader))
    print("Length of the val_loader:", len(args.validation_loader))
    print("Length of the test_loader:", len(args.test_loader))
    
    print('----------------------------------------------------------------')
    
    if check_single_class(args.validation_loader) == True:
        no_hypo.append(k)
        print("skipped, there is no hypo.")
        continue
    elif check_single_class(args.test_loader) == True:
        no_hypo.append(k)
        print("skipped, there is no hypo.")
        continue
    else:
        pass
    
    
    
    print('----------------------------------------------------------------')
    """calculate pos_weight for unbalanced class"""
    pos_class = 0
    neg_class = 0
    for batch_idx,(_,target) in enumerate(args.train_loader):
        pos_class += len(target[np.where(target == 1)])
        neg_class += len(target[np.where(target == 0)])
        
    pos_weight = torch.tensor([neg_class/pos_class], dtype = torch.float32)
    
    print("train 개수:",len(train_indexs)) 
    print("num of class 1:",pos_class) 
    print("num of class 0:",neg_class)
    print(pos_weight)
    args.pos_weight = pos_weight
    
    """---------------------------------------------optimizer---------------------------------------------"""
    args.device = device
    args.model = Classification_model().to(device)
    args.optimizer = optim.Adam(args.model.parameters(), lr=args.lr)
    args.criterion = nn.BCEWithLogitsLoss(pos_weight = args.pos_weight.cuda())
    
    """Train"""
    args.best_acc, args.best_auroc, args.best_epoch, avg_train_losses, avg_valid_losses, Train_ACC, Validation_ACC, Train_AUROC, Validation_AUROC = Model_Train(args)
    
    
    print('----------------------------------------------------------------')
    loss_save_path = args.fold_path +'/Loss.png'

    fig = plt.figure(figsize=(10,8))
    plt.plot(range(1,len(avg_train_losses)+1),avg_train_losses, label='Training Loss')
    plt.plot(range(1,len(avg_valid_losses)+1),avg_valid_losses, label='Validation Loss')


    plt.xlabel('epochs')
    plt.ylabel('loss')

    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(loss_save_path, dpi = 200) 
    #plt.show()
    
    plt.cla()
    plt.clf()
    plt.close()
    
    print('----------------------------------------------------------------')
    acc_save_path = args.fold_path +'/Acc and AUROC.png'

    fig = plt.figure(figsize=(10,8))

    plt.plot (range(1,len(Train_ACC)+1),np.array(Train_ACC)/100, label='Train ACC',alpha = 0.2 )
    plt.plot (range(1,len(Validation_ACC)+1),np.array(Validation_ACC)/100, label='Validation ACC',alpha = 0.2)
    plt.plot (range(1,len(Train_AUROC)+1),np.array(Train_AUROC), label='Train AUROC')
    plt.plot (range(1,len(Validation_AUROC)+1),np.array(Validation_AUROC), label='Validation AUROC')

    plt.vlines(args.best_epoch,0.1,1.0,'r','--')
    plt.text(args.best_epoch, 0.1,' best model',fontsize = 15)

    plt.xlabel('epochs')
    plt.ylabel('acc and auroc')

    plt.xlim(0, len(Train_ACC)+1)
    plt.ylim(0, 1)

    plt.grid(True)
    plt.legend(loc = 'lower right', fancybox = False, edgecolor = 'k', framealpha = 1.0,fontsize = 15)
    plt.tight_layout()
    plt.savefig(acc_save_path, dpi = 200) 
    #plt.show()
    
    plt.cla()
    plt.clf()
    plt.close()
    
    print('----------------------------------------------------------------')
    
    args.test_result = Model_Test(args)
    
    print('----------------------------------------------------------------')
    
    args.test_result_youden_index = Model_Test_Youden_Index(args)
    
    print('----------------------------------------------------------------')
    
    """save"""
    args_copy = args.copy()
    del_list = ['train_loader','validation_loader','test_loader','model','test_result', 'test_result_youden_index']

    for i in range(len(del_list)):
        del args_copy[del_list[i]]

    df_train = pd.DataFrame(args_copy,index = [0]).T
    df_train.to_excel(args.fold_path +'/Result.xlsx',index=True)
    
    df_test = pd.DataFrame(args.test_result,index = [0]).T
    df_test.to_excel(args.fold_path +'/Result_test.xlsx',index=True)
    
    df_test_youden = pd.DataFrame(args.test_result_youden_index,index = [0]).T
    df_test_youden.to_excel(args.fold_path +'/Result_test_youden.xlsx',index=True)
    
    
    print("result was saved.")
    print('----------------------------------------------------------------')
    